<a href="https://colab.research.google.com/github/vrra/FGAN-Build-a-thon/blob/main/Notebooks2023/PDF_parsing_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install all libraries

In [2]:
!pip install pypdf
!pip install langchain
!pip install pandas
!pip install huggingface_hub
!pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00


Imports

In [3]:
import pandas as pd
from huggingface_hub import login
from google.colab import userdata

Now we read the data from the pdf file.

In [ ]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/sample_data/Use-case-AN.pdf")
pages = loader.load_and_split()
simple_data_container = []

for i in range(len(pages)):
  print("Physical Page =",
      i,
      "logical page given by the loader = ",
      pages[i].metadata['page'],
      "of ",
      pages[i].metadata['source'],
      "content = ",
      pages[i].page_content)
  simple_data_container.append(
      [i,
       pages[i].metadata['source'],
       pages[i].page_content])
  print("*****************************")

df = pd.DataFrame(simple_data_container,
                  columns=['page_number', 'document_source', 'page_content'])
df.head(10)
df.to_parquet('/content/sample_data/p1_usecases.parquet')

access the HF hub,

read the dataset from it

check whether a parquet file is already there

if yes, create a df from it
and append the data_container into it.

In [6]:
hf_token=userdata.get('my_hf_write_token')
login(hf_token)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="/content/sample_data/p1_usecases.parquet",
    path_in_repo="p1_usecases.parquet",
    repo_id="vishnuramov/ITU-AN-usecases-1",
    repo_type="dataset",
)